In [3]:
import numpy as np
import scipy
import scipy.sparse as sp
import scipy.stats as stats
from scipy.optimize import minimize
from scipy.spatial.distance import cdist
from scipy.stats import nbinom, poisson, bernoulli

import pandas as pd
import anndata
import scanpy as sc

from multiprocessing import Pool
from joblib import Parallel, delayed

from tqdm import tqdm
from tqdm.notebook import tqdm

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
import numpy as np
import pandas as pd
import anndata
import scipy.sparse as sp
from scipy.spatial.distance import cdist
import scanpy as sc
from joblib import Parallel, delayed
from tqdm import tqdm




import anndata as ad
import numpy as np
from scipy import sparse

import numpy as np
import pandas as pd
import anndata
import scipy.sparse as sp
from scipy.spatial.distance import cdist
import scanpy as sc
from joblib import Parallel, delayed
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import genpareto
import scanpy as sc
from scipy import stats, optimize
from joblib import Parallel, delayed
from tqdm import tqdm
import pandas as pd
import anndata
import scipy.sparse as sp
from scipy.optimize import minimize_scalar
import numpy as np
import scipy.sparse as sp
from scipy.stats import poisson, nbinom, bernoulli
from multiprocessing import Pool
import anndata

In [4]:

def simulate_gene_average_expression(adata, pseudocount=1, n_simulations=1000):
    if sp.issparse(adata.X):
        X = adata.X.toarray()
    else:
        X = adata.X
    gene_totals = X.sum(axis=0)
    gene_totals_pseudo = gene_totals + pseudocount
    total_reads = gene_totals_pseudo.sum()
    gene_probs = gene_totals_pseudo / total_reads
    simulated_totals = np.random.multinomial(int(total_reads), gene_probs, size=n_simulations)
    average_simulated_expression = simulated_totals.mean(axis=0)
    n_cells = X.shape[0]
    average_expression = average_simulated_expression / n_cells
    return dict(zip(adata.var_names, average_expression))

## 基础模型部分

### 模型拟合部分：mean，var拟合以及基因模型拟合

In [5]:
import numpy as np
from scipy import sparse as sp
from scipy.stats import genpareto, ks_2samp

class MeanSimulator:
    def __init__(self, mode="strict", threshold=0.99):
        self.mode = mode
        self.threshold = threshold
        self.shape = None
        self.loc = None
        self.scale = None
        self.best_threshold = None
        self.tail_data = None
        self.original_order = None
        self.best_evaluation = None
        self.original_data = None

    def extract_data(self, X):
        return np.mean(X, axis=0)

    def fit(self, adata):
        if sp.issparse(adata.X):
            X = adata.X.toarray()
        else:
            X = adata.X
        
        self.original_data = self.extract_data(X)
        self.original_order = np.argsort(self.original_data)
        sorted_data = np.sort(self.original_data)

        if self.mode == "strict":
            thresholds = [99.5, 99, 98.5, 98, 97.5, 97,96,95]
            best_score = float('inf')

            for percentile in thresholds:
                current_threshold = np.percentile(sorted_data, percentile)
                main_data = sorted_data[sorted_data <= current_threshold]
                self.tail_data = sorted_data[sorted_data > current_threshold]

                shape, loc, scale = genpareto.fit(main_data)

                n_main = len(main_data)
                n_tail = len(self.tail_data)

                new_main = genpareto.rvs(shape, loc, scale, size=n_main)
                new_tail = np.random.choice(self.tail_data, size=n_tail, replace=True)

                new_samples = np.concatenate([new_main, new_tail])
                new_samples = np.clip(new_samples, np.min(self.original_data), np.max(self.original_data))

                evaluation = self.evaluate_fit(self.original_data, new_samples)

                score = (abs(evaluation["Cohen's d"]) + 
                         evaluation["Relative Error"] + 
                         evaluation["KS Statistic"] + 
                         (1 - evaluation["Correlation"]))

                if score < best_score:
                    best_score = score
                    self.best_threshold = percentile
                    self.shape, self.loc, self.scale = shape, loc, scale
                    self.best_evaluation = evaluation

            print(f"Best threshold: {self.best_threshold}")
            print(f"Best shape: {self.shape}")
            print(f"Best loc: {self.loc}")
            print(f"Best scale: {self.scale}")

        else:
            threshold_value = np.percentile(sorted_data, 99)
            main_data = sorted_data[sorted_data <= threshold_value]
            self.tail_data = sorted_data[sorted_data > threshold_value]

            self.shape, self.loc, self.scale = genpareto.fit(main_data)

        return self

    def simulate(self, n_samples):
        if self.mode == "strict":
            n_main = int(n_samples * self.best_threshold / 100)
        else:
            n_main = int(n_samples * 0.99)  # 使用99%作为非严格模式的默认阈值
        n_tail = n_samples - n_main

        new_main = genpareto.rvs(self.shape, loc=self.loc, scale=self.scale, size=n_main)
        new_tail = np.random.choice(self.tail_data, size=n_tail, replace=True)

        new_samples = np.concatenate([new_main, new_tail])
        new_samples = np.clip(new_samples, np.min(self.original_data), np.max(self.original_data))
        new_samples = np.sort(new_samples)
        
        # 确保模拟的数据保持原来的排序
        simulated_data = np.zeros_like(new_samples)
        simulated_data[self.original_order] = new_samples
        
        return simulated_data

    @staticmethod
    def evaluate_fit(original, generated, quantiles=[0.25, 0.5, 0.75, 0.9, 0.95, 0.99]):
        def cohens_d(x1, x2):
            n1, n2 = len(x1), len(x2)
            var1, var2 = np.var(x1, ddof=1), np.var(x2, ddof=1)
            pooled_se = np.sqrt(((n1 - 1) * var1 + (n2 - 1) * var2) / (n1 + n2 - 2))
            return (np.mean(x1) - np.mean(x2)) / pooled_se

        def relative_error(x1, x2):
            return np.abs(np.mean(x1) - np.mean(x2)) / np.mean(x1)

        # 计算主要指标
        effect_size = cohens_d(original, generated)
        rel_error = relative_error(original, generated)
        ks_stat, _ = ks_2samp(original, generated)
        correlation = np.corrcoef(np.sort(original), np.sort(generated))[0, 1]

        # 计算分位数相对误差
        orig_quant = np.quantile(original, quantiles)
        gen_quant = np.quantile(generated, quantiles)
        quant_rel_errors = np.abs(orig_quant - gen_quant) / orig_quant

        # 评估结果
        results = {
            "Cohen's d": effect_size,
            "Relative Error": rel_error,
            "KS Statistic": ks_stat,
            "Correlation": correlation,
            "Quantile Relative Errors": dict(zip([f"{q*100}th" for q in quantiles], quant_rel_errors))
        }

        # 修改判定标准
        excellent = (abs(effect_size) < 0.05 and rel_error < 0.05 and ks_stat < 0.1 and correlation > 0.95)
        good = (abs(effect_size) < 0.1 and rel_error < 0.15 and ks_stat < 0.15 and correlation > 0.9)
        fair = (abs(effect_size) < 0.2 and rel_error < 0.2 and ks_stat < 0.2 and correlation > 0.8)

        if excellent:
            verdict = "Excellent fit"
        elif good:
            verdict = "Good fit"
        elif fair:
            verdict = "Fair fit"
        else:
            verdict = "Poor fit"

        results["Verdict"] = verdict

        return results

def simulate_gene_means(adata, mode="strict", threshold=0.99):
    simulator = MeanSimulator(mode=mode, threshold=threshold)
    simulator.fit(adata)
    simulated_values = simulator.simulate(adata.n_vars)
    
    result_dict = dict(zip(adata.var_names, simulated_values))
    
    if mode == "strict":
        return result_dict, simulator.best_threshold, simulator.best_evaluation
    else:
        return result_dict

In [6]:
import numpy as np
from scipy import sparse as sp
from scipy.stats import invgamma, ks_2samp
from scipy.optimize import minimize
from scipy.special import gammaln

class IG_VarianceSimulator:
    def __init__(self):
        self.alpha = None
        self.beta = None
        self.threshold = None
        self.tail_data = None
        self.original_order = None
        self.original_data = None
        self.random_state = None

    def extract_data(self, X):
        variances = np.var(X, axis=0, ddof=1)
        variances = np.nan_to_num(variances, nan=np.nanmean(variances))
        variances = np.maximum(variances, 1e-10)
        return variances

    def assess_tail_discreteness(self, tail_data):
        sorted_tail = np.sort(tail_data)
        differences = np.diff(sorted_tail)
        cv = np.std(differences) / np.mean(differences)

        if cv > 2.0:
            return 'discrete'
        elif cv < 1.0:
            return 'smooth'
        else:
            return 'mixed'

    def fit_single_component(self, data):
        try:
            params = invgamma.fit(data, floc=0)
            return params[0], params[2]
        except Exception as e:
            print(f"Single component fitting failed: {str(e)}")
            return 1.0, np.mean(data)  # Fallback values

    def fit(self, adata):
        try:
            if sp.issparse(adata.X):
                X = adata.X.toarray()
            else:
                X = adata.X

            self.original_data = self.extract_data(X)
            if not np.all(np.isfinite(self.original_data)):
                raise ValueError("Data contains non-finite values")

            self.original_order = np.argsort(self.original_data)
            sorted_data = np.sort(self.original_data)

            thresholds = [99.5]
            best_score = float('inf')
            best_evaluation = None
            for percentile in thresholds:
                current_threshold = np.percentile(sorted_data, percentile)
                main_data = sorted_data[sorted_data <= current_threshold]
                tail_data = sorted_data[sorted_data > current_threshold]

                if len(main_data) == 0 or len(tail_data) == 0:
                    print(f"Warning: Empty main_data or tail_data at threshold {percentile}")
                    continue

                alpha, beta = self.fit_single_component(main_data)

                n_main = len(main_data)
                n_tail = len(tail_data)

                new_main = invgamma.rvs(alpha, scale=beta, size=n_main, random_state=self.random_state)

                tail_type = self.assess_tail_discreteness(tail_data)
                if tail_type == 'discrete':
                    new_tail = self.random_state.choice(tail_data, size=n_tail, replace=True)
                else:
                    new_tail = np.interp(
                        np.linspace(0, 1, n_tail),
                        np.linspace(0, 1, len(tail_data)),
                        np.sort(tail_data)
                    )

                new_samples = np.concatenate([new_main, new_tail])
                new_samples = np.clip(new_samples, np.min(self.original_data), np.max(self.original_data))

                evaluation = self.evaluate_fit(self.original_data, new_samples)

                score = (abs(evaluation["Cohen's d"]) + 
                         evaluation["Relative Error"] + 
                         evaluation["KS Statistic"] + 
                         (1 - evaluation["Correlation"]))

                if score < best_score:
                    best_score = score
                    self.threshold = percentile
                    self.alpha, self.beta = alpha, beta
                    best_evaluation = evaluation
                    self.tail_data = tail_data

            if self.threshold is None:
                print("Warning: No valid threshold found, fallback to default.")
                self.threshold = 95  # Default threshold

            return self, best_evaluation

        except Exception as e:
            print(f"Fitting failed: {str(e)}")
            self.alpha, self.beta = self.fit_single_component(self.original_data)
            return self, {"Verdict": "Fallback to single component"}

    def simulate(self, n_samples):
        try:
            if self.threshold is None:
                print("Warning: Threshold is None, using default value of 95.")
                self.threshold = 95

            n_main = int(n_samples * self.threshold / 100)
            n_tail = n_samples - n_main

            new_main = invgamma.rvs(self.alpha, scale=self.beta, size=n_main, random_state=self.random_state)

            tail_type = self.assess_tail_discreteness(self.tail_data)
            if tail_type == 'discrete':
                new_tail = self.random_state.choice(self.tail_data, size=n_tail, replace=True)
            else:
                new_tail = np.interp(
                    np.linspace(0, 1, n_tail),
                    np.linspace(0, 1, len(self.tail_data)),
                    np.sort(self.tail_data)
                )

            new_samples = np.concatenate([new_main, new_tail])
            new_samples = np.clip(new_samples, np.min(self.original_data), np.max(self.original_data))
            new_samples = np.sort(new_samples)

            simulated_data = np.zeros_like(new_samples)
            simulated_data[self.original_order] = new_samples

            return simulated_data

        except Exception as e:
            print(f"Simulation failed: {str(e)}")
            return self.random_state.choice(self.original_data, size=n_samples, replace=True)

    @staticmethod
    def evaluate_fit(original, generated, quantiles=[0.25, 0.5, 0.75, 0.9, 0.95, 0.99, 1]):
        """评估生成的数据与原始数据的拟合度"""
        def cohens_d(x1, x2):
            n1, n2 = len(x1), len(x2)
            var1, var2 = np.var(x1, ddof=1), np.var(x2, ddof=1)
            pooled_se = np.sqrt(((n1 - 1) * var1 + (n2 - 1) * var2) / (n1 + n2 - 2))
            return (np.mean(x1) - np.mean(x2)) / pooled_se

        def relative_error(x1, x2):
            return np.abs(np.mean(x1) - np.mean(x2)) / np.mean(x1)

        effect_size = cohens_d(original, generated)
        rel_error = relative_error(original, generated)
        ks_stat, _ = ks_2samp(original, generated)
        correlation = np.corrcoef(np.sort(original), np.sort(generated))[0, 1]

        orig_quant = np.quantile(original, quantiles)
        gen_quant = np.quantile(generated, quantiles)
        quant_rel_errors = np.abs(orig_quant - gen_quant) / orig_quant

        results = {
            "Cohen's d": effect_size,
            "Relative Error": rel_error,
            "KS Statistic": ks_stat,
            "Correlation": correlation,
            "Quantile Relative Errors": dict(zip([f"{q*100}th" for q in quantiles], quant_rel_errors))
        }

        excellent = (abs(effect_size) < 0.05 and rel_error < 0.05 and ks_stat < 0.1 and correlation > 0.95)
        good = (abs(effect_size) < 0.1 and rel_error < 0.15 and ks_stat < 0.15 and correlation > 0.9)
        fair = (abs(effect_size) < 0.2 and rel_error < 0.2 and ks_stat < 0.2 and correlation > 0.8)

        if excellent:
            verdict = "Excellent fit"
        elif good:
            verdict = "Good fit"
        elif fair:
            verdict = "Fair fit"
        else:
            verdict = "Poor fit"

        results["Verdict"] = verdict

        return results

    def fit_and_simulate(self, adata, n_iterations=30):
        best_simulation = None
        best_evaluation = None
        best_score = float('inf')
        best_threshold = None
        best_alpha = None
        best_beta = None

        for _ in range(n_iterations):
            self.random_state = np.random.RandomState()  # 每次迭代使用新的随机种子
            self, evaluation = self.fit(adata)
            simulated_values = self.simulate(adata.n_vars)
            final_evaluation = self.evaluate_fit(self.original_data, simulated_values)

            score = (abs(final_evaluation["Cohen's d"]) + 
                     final_evaluation["Relative Error"] + 
                     final_evaluation["KS Statistic"] + 
                     (1 - final_evaluation["Correlation"]))

            if score < best_score:
                best_score = score
                best_simulation = simulated_values
                best_evaluation = final_evaluation
                best_threshold = self.threshold
                best_alpha = self.alpha
                best_beta = self.beta

        self.threshold = best_threshold
        self.alpha = best_alpha
        self.beta = best_beta

        return best_simulation, best_evaluation

def simulate_gene_variances_advanced(adata, n_iterations=10):
    simulator = IG_VarianceSimulator()
    simulated_values, final_evaluation = simulator.fit_and_simulate(adata, n_iterations)

    result_dict = dict(zip(adata.var_names, simulated_values))

    print("\nFinal evaluation:")
    print(final_evaluation)

    # 添加更多诊断信息
    print("\nDiagnostic Information:")
    print(f"Original data mean: {np.mean(simulator.original_data)}")
    print(f"Original data variance: {np.var(simulator.original_data)}")
    print(f"Simulated data mean: {np.mean(simulated_values)}")
    print(f"Simulated data variance: {np.var(simulated_values)}")
    print(f"Best threshold: {simulator.threshold}")
    print(f"Best alpha: {simulator.alpha}")
    print(f"Best beta: {simulator.beta}")

    return result_dict, simulator.threshold, final_evaluation

In [7]:
def calc_zero_proportion(X):
    return np.mean(X == 0, axis=0)

def genpareto_fit_zero_p(filtered_adata, mode="strict", threshold=0.99):
    if sp.issparse(filtered_adata.X):
        filtered_adata.X = filtered_adata.X.A

    zero_proportion = calc_zero_proportion(filtered_adata.X)
    zero_proportion = 1- zero_proportion

    def transform_data(data, epsilon=1e-10):
        return -np.log(1 - np.clip(data, 0, 1-epsilon))

    def inverse_transform(data):
        return 1 - np.exp(-data)

    def fit_and_sample(data, n_samples):
        transformed_data = transform_data(data)
        shape, loc, scale = genpareto.fit(transformed_data)
        samples = genpareto.rvs(shape, loc, scale, size=n_samples)
        return inverse_transform(samples)

    def evaluate_fit(original, new):
        ks_stat, _ = ks_2samp(original, new)
        return {
            "KS Statistic": ks_stat,
            "Mean Difference": np.mean(original) - np.mean(new),
            "Std Difference": np.std(original) - np.std(new),
            "Correlation": np.corrcoef(original, new)[0, 1]
        }

    if mode == "strict":
        thresholds = [0.995, 0.99, 0.985, 0.98, 0.975]
        best_score = float('inf')
        best_threshold = None
        best_samples = None
        best_evaluation = None

        for percentile in thresholds:
            threshold = np.percentile(zero_proportion, percentile * 100)
            main_data = zero_proportion[zero_proportion <= threshold]
            tail_data = zero_proportion[zero_proportion > threshold]

            n_main = len(main_data)
            n_tail = len(tail_data)

            new_main = fit_and_sample(main_data, n_main)
            new_tail = np.random.choice(tail_data, size=n_tail, replace=True)

            new_samples = np.concatenate([new_main, new_tail])
            new_samples = np.clip(new_samples, 0, 1)

            evaluation = evaluate_fit(zero_proportion, new_samples)

            score = (abs(evaluation["Mean Difference"]) + 
                     abs(evaluation["Std Difference"]) + 
                     evaluation["KS Statistic"] + 
                     (1 - evaluation["Correlation"]))

            if score < best_score:
                best_score = score
                best_threshold = percentile
                best_samples = new_samples
                best_evaluation = evaluation

        original_order = np.argsort(zero_proportion)
        best_samples = best_samples[original_order]

        return 1-best_samples, best_threshold, best_evaluation

    else:
        threshold = np.percentile(zero_proportion, threshold * 100)
        main_data = zero_proportion[zero_proportion <= threshold]
        tail_data = zero_proportion[zero_proportion > threshold]

        n_main = len(main_data)
        n_tail = len(tail_data)

        new_main = fit_and_sample(main_data, n_main)
        new_tail = np.random.choice(tail_data, size=n_tail, replace=True)

        new_samples = np.concatenate([new_main, new_tail])
        new_samples = np.clip(new_samples, 0, 1)

        original_order = np.argsort(zero_proportion)
        new_samples = new_samples[original_order]

        return 1-new_samples
    


def simulate_gene_pis(adata, mode="strict", threshold=0.99):
    simulated_values, best_threshold, best_evaluation = genpareto_fit_zero_p(adata, mode=mode, threshold=threshold)
    
    result_dict = dict(zip(adata.var_names, simulated_values))
    
    if mode == "strict":
        return result_dict, best_threshold, best_evaluation
    else:
        return result_dict

In [8]:
def nb_loglikelihood_fixed_mu(r, x, mu):
    p = r / (r + mu)
    return -np.sum(stats.nbinom.logpmf(x, r, p))

def zinb_loglikelihood_fixed_mu(params, x, mu):
    pi, r = params
    p = r / (r + mu)
    x_is_zero = (x == 0)
    ll_zero = x_is_zero * np.log(pi + (1 - pi) * stats.nbinom.pmf(0, r, p))
    ll_nonzero = ~x_is_zero * (np.log(1 - pi) + stats.nbinom.logpmf(x, r, p))
    return -np.sum(ll_zero + ll_nonzero)

def zip_loglikelihood_fixed_mu(pi, x, mu):
    x_is_zero = (x == 0)
    ll_zero = x_is_zero * np.log(pi + (1 - pi) * np.exp(-mu))
    ll_nonzero = ~x_is_zero * (np.log(1 - pi) + stats.poisson.logpmf(x, mu))
    return -np.sum(ll_zero + ll_nonzero)

def poisson_loglikelihood(mu, x):
    return -np.sum(stats.poisson.logpmf(x, mu))

def fit_with_simulated_mean_and_var(gene, simulated_mean, simulated_var, maxiter=100):
    mu = simulated_mean
    
    # 计算负二项分布参数
    if simulated_var < simulated_mean:  # 如果方差小于均值，使用泊松分布
        return [0, np.inf, mu, "Poisson"]
    else:
        p = (simulated_var - mu) / simulated_var
        r = mu * (1 - p) / p
        
        # 计算各个模型的似然值
        ll_nb = -nb_loglikelihood_fixed_mu(r, gene, mu)
        
        result_zinb = minimize(zinb_loglikelihood_fixed_mu, [0.5, r], 
                             args=(gene, mu), 
                             bounds=[(1e-6, 1-1e-6), (1e-6, 1e6)])
        pi_zinb, r_zinb = result_zinb.x
        ll_zinb = -result_zinb.fun

        result_zip = minimize_scalar(zip_loglikelihood_fixed_mu, 
                                   args=(gene, mu), 
                                   bounds=(1e-6, 1-1e-6), 
                                   method='bounded')
        pi_zip = result_zip.x
        ll_zip = -result_zip.fun

        ll_poisson = -poisson_loglikelihood(mu, gene)

        # 计算AIC
        aic_nb = 2 * 2 - 2 * ll_nb
        aic_zinb = 2 * 3 - 2 * ll_zinb
        aic_zip = 2 * 2 - 2 * ll_zip
        aic_poisson = 2 * 1 - 2 * ll_poisson
        
        aics = [aic_nb, aic_zinb, aic_zip, aic_poisson]
        best_model_idx = np.argmin(aics)

        if best_model_idx == 0:
            return [0, r, mu, "NB"]
        elif best_model_idx == 1:
            return [pi_zinb, r_zinb, mu, "ZINB"]
        elif best_model_idx == 2:
            return [pi_zip, np.inf, mu, "ZIP"]
        else:
            return [0, np.inf, mu, "Poisson"]

def fit_marginal_model_with_simulated_params(adata, simulated_means, simulated_vars, 
                                           min_nonzero_num=2, maxiter=500, n_jobs=-1):
    if not isinstance(adata, anndata.AnnData):
        raise ValueError("Input adata should be an AnnData object")
    
    if sp.issparse(adata.X):
        x = adata.X.toarray()
    else:
        x = adata.X
    
    gene_names = adata.var_names.tolist()
    n, p = x.shape
    
    if len(simulated_means) != p or len(simulated_vars) != p:
        raise ValueError("Length of simulated parameters does not match number of genes")
    
    gene_zero_prop = 1 - np.sum(x > 0, axis=0) / n
    gene_sel1 = np.where(gene_zero_prop < 1 - min_nonzero_num / n)[0]
    gene_sel2 = np.setdiff1d(np.arange(p), gene_sel1)
    
    if len(gene_sel1) == 0:
        print("Warning: No genes selected for fitting models.")
        return None
    
    results = Parallel(n_jobs=n_jobs)(
        delayed(fit_with_simulated_mean_and_var)(
            x[:, i], 
            simulated_means[gene_names[i]], 
            simulated_vars[gene_names[i]], 
            maxiter
        )
        for i in tqdm(gene_sel1, desc="Fitting models")
    )
    
    params_df = pd.DataFrame(results, 
                           index=[gene_names[i] for i in gene_sel1], 
                           columns=['pi0', 'theta', 'mu', 'model_selected'])
    
    model_params = {
        'gene_sel1': {i: gene_names[i] for i in gene_sel1},
        'gene_sel2': {i: gene_names[i] for i in gene_sel2},
        'marginal_param1': params_df[['pi0', 'theta', 'mu']].values.tolist(),
        'model_selected': params_df['model_selected'].tolist(),
        'min_nonzero_num': min_nonzero_num,
        'n_cell': n,
        'n_read': np.sum(x)
    }
    
    return model_params

In [9]:
# import numpy as np
# from scipy import stats
# from scipy.optimize import minimize, minimize_scalar
# import pandas as pd
# from joblib import Parallel, delayed
# from tqdm import tqdm
# import anndata
# from scipy import sparse as sp

# def nb_loglikelihood_fixed_mu(r, x, mu):
#     p = r / (r + mu)
#     return -np.sum(stats.nbinom.logpmf(x, r, p))

# def zinb_loglikelihood_fixed_mu(params, x, mu):
#     pi, r = params
#     p = r / (r + mu)
#     x_is_zero = (x == 0)
#     ll_zero = x_is_zero * np.log(pi + (1 - pi) * stats.nbinom.pmf(0, r, p))
#     ll_nonzero = ~x_is_zero * (np.log(1 - pi) + stats.nbinom.logpmf(x, r, p))
#     return -np.sum(ll_zero + ll_nonzero)

# def zip_loglikelihood_fixed_mu(pi, x, mu):
#     x_is_zero = (x == 0)
#     ll_zero = x_is_zero * np.log(pi + (1 - pi) * np.exp(-mu))
#     ll_nonzero = ~x_is_zero * (np.log(1 - pi) + stats.poisson.logpmf(x, mu))
#     return -np.sum(ll_zero + ll_nonzero)

# def poisson_loglikelihood(mu, x):
#     return -np.sum(stats.poisson.logpmf(x, mu))

# def fit_with_simulated_params(gene, simulated_mean, simulated_var, simulated_pi, maxiter=100):
#     mu = simulated_mean
#     pi = simulated_pi
    
#     # 计算负二项分布参数
#     if simulated_var < simulated_mean:  # 如果方差小于均值，使用泊松分布
#         return [pi, np.inf, mu, "ZIP"]
#     else:
#         p = (simulated_var - mu) / simulated_var
#         r = mu * (1 - p) / p
        
#         # 计算各个模型的似然值
#         ll_nb = -nb_loglikelihood_fixed_mu(r, gene, mu)
        
#         result_zinb = minimize(zinb_loglikelihood_fixed_mu, [pi, r], 
#                              args=(gene, mu), 
#                              bounds=[(1e-6, 1-1e-6), (1e-6, 1e6)])
#         pi_zinb, r_zinb = result_zinb.x
#         ll_zinb = -result_zinb.fun

#         ll_zip = -zip_loglikelihood_fixed_mu(pi, gene, mu)

#         ll_poisson = -poisson_loglikelihood(mu, gene)

#         # 计算AIC
#         aic_nb = 2 * 2 - 2 * ll_nb
#         aic_zinb = 2 * 3 - 2 * ll_zinb
#         aic_zip = 2 * 2 - 2 * ll_zip
#         aic_poisson = 2 * 1 - 2 * ll_poisson
        
#         aics = [aic_nb, aic_zinb, aic_zip, aic_poisson]
#         best_model_idx = np.argmin(aics)

#         if best_model_idx == 0:
#             return [0, r, mu, "NB"]
#         elif best_model_idx == 1:
#             return [pi_zinb, r_zinb, mu, "ZINB"]
#         elif best_model_idx == 2:
#             return [pi, np.inf, mu, "ZIP"]
#         else:
#             return [0, np.inf, mu, "Poisson"]

# def fit_marginal_model_with_simulated_params(adata, simulated_means, simulated_vars, simulated_pis,
#                                            min_nonzero_num=2, maxiter=500, n_jobs=-1):
#     if not isinstance(adata, anndata.AnnData):
#         raise ValueError("Input adata should be an AnnData object")
    
#     if sp.issparse(adata.X):
#         x = adata.X.toarray()
#     else:
#         x = adata.X
    
#     gene_names = adata.var_names.tolist()
#     n, p = x.shape
    
#     if len(simulated_means) != p or len(simulated_vars) != p or len(simulated_pis) != p:
#         raise ValueError("Length of simulated parameters does not match number of genes")
    
#     gene_zero_prop = 1 - np.sum(x > 0, axis=0) / n
#     gene_sel1 = np.where(gene_zero_prop < 1 - min_nonzero_num / n)[0]
#     gene_sel2 = np.setdiff1d(np.arange(p), gene_sel1)
    
#     if len(gene_sel1) == 0:
#         print("Warning: No genes selected for fitting models.")
#         return None
    
#     results = Parallel(n_jobs=n_jobs)(
#         delayed(fit_with_simulated_params)(
#             x[:, i], 
#             simulated_means[gene_names[i]], 
#             simulated_vars[gene_names[i]],
#             simulated_pis[gene_names[i]],
#             maxiter
#         )
#         for i in tqdm(gene_sel1, desc="Fitting models")
#     )
    
#     params_df = pd.DataFrame(results, 
#                            index=[gene_names[i] for i in gene_sel1], 
#                            columns=['pi0', 'theta', 'mu', 'model_selected'])
    
#     model_params = {
#         'gene_sel1': {i: gene_names[i] for i in gene_sel1},
#         'gene_sel2': {i: gene_names[i] for i in gene_sel2},
#         'marginal_param1': params_df[['pi0', 'theta', 'mu']].values.tolist(),
#         'model_selected': params_df['model_selected'].tolist(),
#         'min_nonzero_num': min_nonzero_num,
#         'n_cell': n,
#         'n_read': np.sum(x)
#     }
    
#     return model_params

In [10]:

def simulate_gene(iter, gene_names, adata, model_params, rr):
    gene_name = gene_names[iter]
    if gene_name in adata.var_names:
        gene_expr = adata[:, gene_name].X.toarray().flatten()

        print(f"\nSimulating gene: {gene_name}")
        print(f"Original stats - Min: {gene_expr.min():.2f}, Max: {gene_expr.max():.2f}, Mean: {gene_expr.mean():.2f}")

        original_order = np.argsort(gene_expr)
        param = model_params['marginal_param1'][iter]
        model_type = model_params['model_selected'][iter]

        param = [float(p) if p != 'inf' else np.inf for p in param]

        try:
            if model_type == 'Poisson':
                lambda_param = param[2] * rr
                sim_raw_expr = poisson.rvs(lambda_param, size=adata.shape[0])
            elif model_type == 'NB':
                r_param = param[1]
                if np.isinf(r_param):
                    lambda_param = param[2] * rr
                    sim_raw_expr = poisson.rvs(lambda_param, size=adata.shape[0])
                else:
                    p_param = r_param / (r_param + param[2] * rr)
                    r_param = np.maximum(r_param, 1e-8)
                    p_param = np.clip(p_param, 1e-8, 1 - 1e-8)
                    sim_raw_expr = nbinom.rvs(r_param, p_param, size=adata.shape[0])
            elif model_type == 'ZIP':
                pi0 = param[0]
                lambda_param = param[2] * rr
                zero_mask = bernoulli.rvs(pi0, size=adata.shape[0])
                sim_raw_expr = poisson.rvs(lambda_param, size=adata.shape[0]) * (1 - zero_mask)
            elif model_type == 'ZINB':
                pi0 = param[0]
                r_param = param[1]
                p_param = r_param / (r_param + param[2] * rr)

                if r_param <= 0 or not (0 < p_param < 1):
                    raise ValueError(f"Invalid parameters for nbinom: r = {r_param}, p = {p_param}")
                
                zero_mask = bernoulli.rvs(pi0, size=adata.shape[0])
                sim_raw_expr = nbinom.rvs(r_param, p_param, size=adata.shape[0]) * (1 - zero_mask)
            else:
                raise ValueError(f"Unknown model type: {model_type}")
            
        except Exception as e:
            print(f"Warning: Error simulating gene {gene_name} with {model_type} model: {e}")
            print("Falling back to Poisson distribution with mean as lambda")
            # 使用原始数据的平均值作为 Poisson 分布的参数
            mean_expr = np.mean(gene_expr)
            sim_raw_expr = poisson.rvs(mean_expr, size=adata.shape[0])
        
        try:
            sim_order = np.argsort(sim_raw_expr)
            final_expr = np.zeros_like(gene_expr)
            final_expr[original_order] = sim_raw_expr[sim_order]

            top_10_percent = np.percentile(gene_expr, 90)
            original_high = gene_expr > top_10_percent
            final_high = final_expr > top_10_percent
            overlap = np.sum(original_high & final_high) / np.sum(original_high)
            print(f"Top 10% preservation: {overlap * 100:.2f}%")

            correlation = np.corrcoef(gene_expr, final_expr)[0, 1]
            print(f"Correlation: {correlation:.4f}")

            return final_expr
        except Exception as e:
            print(f"Error in post-processing for gene {gene_name}: {e}")
            # 如果后处理也失败，直接返回 Poisson 模拟结果
            return sim_raw_expr
    else:
        return np.zeros(adata.shape[0])

def srtsim_count_single(model_params, adata, rr=1, breaktie='random', num_cores=1):
    p1 = len(model_params['gene_sel1'])
    p2 = len(model_params['gene_sel2'])
    gene_names = [None] * (p1 + p2)

    for idx, (i, gene_name) in enumerate(model_params['gene_sel1'].items()):
        if idx < p1:
            gene_names[idx] = gene_name
        else:
            raise IndexError(f"Index {idx} out of range for gene_sel1")

    for idx, (i, gene_name) in enumerate(model_params['gene_sel2'].items()):
        if idx + p1 < p1 + p2:
            gene_names[idx + p1] = gene_name
        else:
            raise IndexError(f"Index {idx + p1} out of range for gene_sel2")

    if p2 > 0:
        adata = adata[:, ~adata.var_names.isin(model_params['gene_sel2'])]

    num_loc, num_genes = adata.shape
    result = np.zeros((p1 + p2, num_loc), dtype=float)

    if p1 > 0:
        for iter in range(p1):
            result[iter, :] = simulate_gene(iter, gene_names, adata, model_params, rr)

    return result


def srtsim_remain_simulate_count(simsrt, adata, breaktie='random', rrr=None, num_cores=8, verbose=False):
    if simsrt.simcolData is None:
        simsrt.simcolData = simsrt.refcolData.copy()

    oldnum_loc = simsrt.refcolData.shape[0]
    newnum_loc = simsrt.simcolData.shape[0]
    param_res = simsrt.EstParam[0] 

    # Calculate total counts in the old data
    total_count_old = adata.obs['total_counts'].sum()
    total_count_new = total_count_old  
    r = (total_count_new / newnum_loc) / (total_count_old / oldnum_loc) if rrr is None else rrr

    if verbose:
        print(f"The ratio between the seqdepth per location is: {r}")

    # Generate the simulated count matrix
    rawcount = srtsim_count_single(model_params=param_res, adata=adata, rr=r, breaktie=breaktie, num_cores=num_cores)
    

    # Handle all-zero genes
    all_zero_idx = np.where(np.sum(rawcount, axis=1) == 0)[0]
    if len(all_zero_idx) > 0:
        for idx in all_zero_idx:
            nonzero_idx = np.random.choice(newnum_loc, 1)[0]
            rawcount[idx, nonzero_idx] = 1

    outcount = np.round(rawcount).astype(int)
    simsrt.simCounts = sp.csr_matrix(outcount)

    return simsrt

In [11]:
import scanpy as sc
import numpy as np

def run_simulation_tissue(adata, mode="strict", threshold=0.99):
    adata.var_names_make_unique()
    adata.obs_names_make_unique()
    sc.pp.calculate_qc_metrics(adata, inplace=True)

    all_genes = adata.var_names.tolist()
    print(adata)

    # 模拟均值
    # simulated_means, mean_threshold, mean_evaluation = simulate_gene_means(adata, mode=mode, threshold=threshold)
    # print(f"Mean simulation - Best threshold: {mean_threshold}")
    # print(f"Mean simulation - Evaluation: {mean_evaluation}")

    simulated_means = simulate_gene_average_expression(adata)
    # 模拟方差
    simulated_vars, var_evaluation= simulate_gene_variances_advanced(adata)
    

    print(f"Variance simulation - Evaluation: {var_evaluation}")

    print(f"Number of simulated genes: {len(simulated_means)}")
    # 使用模拟的参数拟合边际模型
    model_params = fit_marginal_model_with_simulated_params(
        adata, 
        simulated_means, 
        simulated_vars, 
        min_nonzero_num=2, 
        maxiter=500, 
        n_jobs=-1
    )

    # 添加模拟评估结果到返回的字典中
    model_params['simulation_evaluation'] = {
        'variance': var_evaluation,

    }

    return model_params



class SimSRT:
    def __init__(self, adata, model_params):
        self.refCounts = adata.to_df()  
        self.refcolData = adata.obs.copy()  
        self.simcolData = None
        self.EstParam = [model_params]
        self.simCounts = None

In [10]:
adata = sc.read_h5ad("/Users/chen_yiru/Desktop/simulation/data/raw/Sample_data_151676.h5ad")



## tissue-base 最基础的模拟测试

In [61]:
model_params = run_simulation_tissue(adata)
simsrt = SimSRT(adata, model_params)

simulated_simsrt = srtsim_remain_simulate_count(simsrt, adata, num_cores=8, verbose=True)

simulated_counts = simulated_simsrt.simCounts

if simulated_counts.shape != adata.shape:
    print(f"Warning: simulated_counts shape {simulated_counts.shape} does not match adata shape {adata.shape}")
    if simulated_counts.shape == (adata.shape[1], adata.shape[0]):
        simulated_counts = simulated_counts.T  
    elif simulated_counts.shape != adata.shape:
        raise ValueError("Cannot adjust simulated_counts shape to match adata shape")

simulated_adata = anndata.AnnData(
    X=simulated_counts,
    obs=adata.obs.copy(),
    var=adata.var.copy(),
    obsm={'spatial': adata.obsm['spatial']}
)

simulated_adata.obs['total_counts'] = simulated_adata.X.sum(axis=1)
simulated_adata.obs['n_genes'] = (simulated_adata.X > 0).sum(axis=1)

AnnData object with n_obs × n_vars = 2701 × 10778
    obs: 'in_tissue', 'array_row', 'array_col', 'imagerow', 'imagecol', 'sum_umi', 'sum_gene', 'subject', 'position', 'replicate', 'discard', 'cell_count', 'layer_guess', 'layer_guess_reordered', 'layer_guess_reordered_short', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'n_counts'
    var: 'gene_ids', 'feature_types', 'genome', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'n_counts'
    uns: 'layer_guess_reordered_colors'
    obsm: 'rgb', 'spatial'

Final evaluation:
{"Cohen's d": -0.0006861767930373992, 'Relative Error': 0.019799040027003077, 'KS Statistic': 0.029504546298014467, 'Correlation': 0.9762400072371336, 'Quantile Relative Errors': {'25.0th': 0.05552662590941831, '50.0th': 0.005613150791302

KeyboardInterrupt: 

In [44]:
simulated_adata.write_h5ad("/Users/chen_yiru/Desktop/simulation/data/simulated/simulated_Sample_data_151673.h5ad")

## domain-based 模拟测试

In [25]:
import numpy as np
from scipy import sparse as sp
from scipy.stats import genpareto, ks_2samp
from scipy.optimize import minimize
from scipy.special import gammaln

class GPD_VarianceSimulator:
    def __init__(self):
        self.c = None  # shape parameter
        self.loc = None  # location parameter
        self.scale = None  # scale parameter
        self.threshold = None
        self.tail_data = None
        self.original_order = None
        self.original_data = None
        self.random_state = None

    def extract_data(self, X):
        variances = np.var(X, axis=0, ddof=1)
        variances = np.nan_to_num(variances, nan=np.nanmean(variances))
        variances = np.maximum(variances, 1e-10)
        return variances

    def assess_tail_discreteness(self, tail_data):
        sorted_tail = np.sort(tail_data)
        differences = np.diff(sorted_tail)
        cv = np.std(differences) / np.mean(differences)

        if cv > 2.0:
            return 'discrete'
        elif cv < 1.0:
            return 'smooth'
        else:
            return 'mixed'

    def fit_single_component(self, data):
        try:
            params = genpareto.fit(data)
            return params[0], params[1], params[2]  # c, loc, scale
        except Exception as e:
            print(f"Single component fitting failed: {str(e)}")
            return 0.1, np.min(data), np.std(data)  # Fallback values

    def fit(self, adata):
        try:
            if sp.issparse(adata.X):
                X = adata.X.toarray()
            else:
                X = adata.X

            self.original_data = self.extract_data(X)
            if not np.all(np.isfinite(self.original_data)):
                raise ValueError("Data contains non-finite values")

            self.original_order = np.argsort(self.original_data)
            sorted_data = np.sort(self.original_data)

            thresholds = [99.5]
            best_score = float('inf')
            best_evaluation = None
            for percentile in thresholds:
                current_threshold = np.percentile(sorted_data, percentile)
                main_data = sorted_data[sorted_data <= current_threshold]
                tail_data = sorted_data[sorted_data > current_threshold]

                if len(main_data) == 0 or len(tail_data) == 0:
                    print(f"Warning: Empty main_data or tail_data at threshold {percentile}")
                    continue

                c, loc, scale = self.fit_single_component(main_data)

                n_main = len(main_data)
                n_tail = len(tail_data)

                new_main = genpareto.rvs(c, loc=loc, scale=scale, size=n_main, random_state=self.random_state)

                tail_type = self.assess_tail_discreteness(tail_data)
                if tail_type == 'discrete':
                    new_tail = self.random_state.choice(tail_data, size=n_tail, replace=True)
                else:
                    new_tail = np.interp(
                        np.linspace(0, 1, n_tail),
                        np.linspace(0, 1, len(tail_data)),
                        np.sort(tail_data)
                    )

                new_samples = np.concatenate([new_main, new_tail])
                new_samples = np.clip(new_samples, np.min(self.original_data), np.max(self.original_data))

                evaluation = self.evaluate_fit(self.original_data, new_samples)

                score = (abs(evaluation["Cohen's d"]) + 
                         evaluation["Relative Error"] + 
                         evaluation["KS Statistic"] + 
                         (1 - evaluation["Correlation"]))

                if score < best_score:
                    best_score = score
                    self.threshold = percentile
                    self.c, self.loc, self.scale = c, loc, scale
                    best_evaluation = evaluation
                    self.tail_data = tail_data

            if self.threshold is None:
                print("Warning: No valid threshold found, fallback to default.")
                self.threshold = 95  # Default threshold

            return self, best_evaluation

        except Exception as e:
            print(f"Fitting failed: {str(e)}")
            self.c, self.loc, self.scale = self.fit_single_component(self.original_data)
            return self, {"Verdict": "Fallback to single component"}

    def simulate(self, n_samples):
        try:
            if self.threshold is None:
                print("Warning: Threshold is None, using default value of 95.")
                self.threshold = 95

            n_main = int(n_samples * self.threshold / 100)
            n_tail = n_samples - n_main

            new_main = genpareto.rvs(self.c, loc=self.loc, scale=self.scale, size=n_main, random_state=self.random_state)

            tail_type = self.assess_tail_discreteness(self.tail_data)
            if tail_type == 'discrete':
                new_tail = self.random_state.choice(self.tail_data, size=n_tail, replace=True)
            else:
                new_tail = np.interp(
                    np.linspace(0, 1, n_tail),
                    np.linspace(0, 1, len(self.tail_data)),
                    np.sort(self.tail_data)
                )

            new_samples = np.concatenate([new_main, new_tail])
            new_samples = np.clip(new_samples, np.min(self.original_data), np.max(self.original_data))
            new_samples = np.sort(new_samples)

            simulated_data = np.zeros_like(new_samples)
            simulated_data[self.original_order] = new_samples

            return simulated_data

        except Exception as e:
            print(f"Simulation failed: {str(e)}")
            return self.random_state.choice(self.original_data, size=n_samples, replace=True)


    @staticmethod
    def evaluate_fit(original, generated, quantiles=[0.25, 0.5, 0.75, 0.9, 0.95, 0.99, 1]):
        """评估生成的数据与原始数据的拟合度"""
        def cohens_d(x1, x2):
            n1, n2 = len(x1), len(x2)
            var1, var2 = np.var(x1, ddof=1), np.var(x2, ddof=1)
            pooled_se = np.sqrt(((n1 - 1) * var1 + (n2 - 1) * var2) / (n1 + n2 - 2))
            return (np.mean(x1) - np.mean(x2)) / pooled_se

        def relative_error(x1, x2):
            return np.abs(np.mean(x1) - np.mean(x2)) / np.mean(x1)

        effect_size = cohens_d(original, generated)
        rel_error = relative_error(original, generated)
        ks_stat, _ = ks_2samp(original, generated)
        correlation = np.corrcoef(np.sort(original), np.sort(generated))[0, 1]

        orig_quant = np.quantile(original, quantiles)
        gen_quant = np.quantile(generated, quantiles)
        quant_rel_errors = np.abs(orig_quant - gen_quant) / orig_quant

        results = {
            "Cohen's d": effect_size,
            "Relative Error": rel_error,
            "KS Statistic": ks_stat,
            "Correlation": correlation,
            "Quantile Relative Errors": dict(zip([f"{q*100}th" for q in quantiles], quant_rel_errors))
        }

        excellent = (abs(effect_size) < 0.05 and rel_error < 0.05 and ks_stat < 0.1 and correlation > 0.95)
        good = (abs(effect_size) < 0.1 and rel_error < 0.15 and ks_stat < 0.15 and correlation > 0.9)
        fair = (abs(effect_size) < 0.2 and rel_error < 0.2 and ks_stat < 0.2 and correlation > 0.8)

        if excellent:
            verdict = "Excellent fit"
        elif good:
            verdict = "Good fit"
        elif fair:
            verdict = "Fair fit"
        else:
            verdict = "Poor fit"

        results["Verdict"] = verdict

        return results

    def fit_and_simulate(self, adata, n_iterations=30):
        best_simulation = None
        best_evaluation = None
        best_score = float('inf')
        best_threshold = None
        best_c = None
        best_loc = None
        best_scale = None

        for _ in range(n_iterations):
            self.random_state = np.random.RandomState()  # 每次迭代使用新的随机种子
            self, evaluation = self.fit(adata)
            simulated_values = self.simulate(adata.n_vars)
            final_evaluation = self.evaluate_fit(self.original_data, simulated_values)

            score = (abs(final_evaluation["Cohen's d"]) + 
                    final_evaluation["Relative Error"] + 
                    final_evaluation["KS Statistic"] + 
                    (1 - final_evaluation["Correlation"]))

            if score < best_score:
                best_score = score
                best_simulation = simulated_values
                best_evaluation = final_evaluation
                best_threshold = self.threshold
                best_c = self.c
                best_loc = self.loc
                best_scale = self.scale

        self.threshold = best_threshold
        self.c = best_c
        self.loc = best_loc
        self.scale = best_scale

        return best_simulation, best_evaluation


def simulate_gene_variances_domain(adata, n_iterations=10):
    simulator = GPD_VarianceSimulator()
    simulated_values, final_evaluation = simulator.fit_and_simulate(adata, n_iterations)

    result_dict = dict(zip(adata.var_names, simulated_values))
    return result_dict, simulator.threshold

In [12]:
import numpy as np
import scipy.sparse as sp
import scanpy as sc
from scipy.stats import invgamma, ks_2samp
from scipy.special import logsumexp
import warnings
warnings.filterwarnings('ignore')
class IG_VarianceSimulator:
    def __init__(self):
        self.components = []
        self.weights = None
        self.alphas = None
        self.betas = None
        self.threshold = None
        self.tail_data = None
        self.original_order = None
        self.original_data = None
        self.random_state = None
        self.n_components = None

    def fit_mixture_components(self, data, max_components=10, max_iter=50):
        """使用EM算法拟合混合逆伽马分布"""
        best_bic = float('inf')
        best_params = None
        best_n_components = None
        
        # 确保数据是有效的
        data = np.array(data)
        data = data[data > 0]  # 确保所有值都是正数
        
        # 预计算数据统计量
        data_mean = np.mean(data)
        data_std = np.std(data)
        
        # 从最简单的模型开始
        for n_components in range(1, max_components + 1):
            try:
                # 更保守的初始化
                weights = np.ones(n_components) / n_components
                alphas = np.ones(n_components) * 2  # 从更安全的值开始
                betas = np.ones(n_components) * data_mean
                
                prev_ll = -np.inf
                for iter_count in range(max_iter):
                    # E步：计算责任
                    resp = np.zeros((len(data), n_components))
                    for j in range(n_components):
                        resp[:, j] = weights[j] * invgamma.pdf(data, alphas[j], scale=betas[j])
                    resp_sum = resp.sum(axis=1, keepdims=True)
                    resp_sum[resp_sum == 0] = 1e-300  # 避免除以零
                    resp /= resp_sum
                    
                    # M步：更新参数
                    weights = resp.mean(axis=0)
                    weights /= weights.sum()  # 确保权重和为1
                    
                    for j in range(n_components):
                        try:
                            mask = resp[:, j] > 0.01  # 降低阈值
                            if np.sum(mask) > 3:  # 确保有足够的数据点
                                component_data = data[mask]
                                # 使用更稳定的参数估计
                                params = invgamma.fit(component_data, floc=0)
                                alphas[j] = max(1.1, min(10, params[0]))  # 限制alpha的范围
                                betas[j] = max(data_mean/10, min(data_mean*10, params[2]))  # 限制beta的范围
                        except:
                            continue
                    
                    # 计算当前似然
                    current_ll = np.sum(np.log(np.sum(weights[None, :] * 
                                    invgamma.pdf(data[:, None], alphas[None, :], scale=betas[None, :]), axis=1)))
                    
                    # 检查收敛
                    if abs(current_ll - prev_ll) < 1e-3 * abs(prev_ll):
                        break
                    prev_ll = current_ll
                
                # 计算BIC
                bic = -2 * current_ll + np.log(len(data)) * (3 * n_components - 1)
                
                if bic < best_bic:
                    best_bic = bic
                    best_params = (weights, alphas, betas)
                    best_n_components = n_components
                    
            except Exception as e:
                print(f"Component {n_components} fitting failed: {str(e)}")
                continue
        
        if best_params is None:
            # 如果所有拟合都失败，返回简单的单组件拟合
            return (np.array([1.0]), np.array([2.0]), np.array([data_mean])), 1
        
        return best_params, best_n_components

    def fit(self, adata):
        try:
            if sp.issparse(adata.X):
                X = adata.X.toarray()
            else:
                X = adata.X

            self.original_data = self.extract_data(X)
            if not np.all(np.isfinite(self.original_data)):
                raise ValueError("Data contains non-finite values")

            # 确保数据是有效的
            if len(self.original_data) == 0:
                raise ValueError("No valid data after preprocessing")

            self.original_order = np.argsort(self.original_data)
            sorted_data = np.sort(self.original_data)

            # 尝试更多的阈值
            thresholds = [95, 97.5, 99, 99.5]
            best_score = float('inf')
            best_params = None
            best_evaluation = None
            
            for threshold in thresholds:
                threshold_value = np.percentile(sorted_data, threshold)
                main_data = sorted_data[sorted_data <= threshold_value]
                tail_data = sorted_data[sorted_data > threshold_value]

                if len(main_data) < 10 or len(tail_data) < 5:
                    continue

                try:
                    (weights, alphas, betas), n_components = self.fit_mixture_components(main_data)
                    
                    self.weights, self.alphas, self.betas = weights, alphas, betas
                    self.n_components = n_components
                    self.threshold = threshold
                    self.tail_data = tail_data
                    
                    simulated = self.simulate(len(self.original_data))
                    evaluation = self.evaluate_fit(self.original_data, simulated)
                    
                    score = (abs(evaluation["Cohen's d"]) + 
                            evaluation["Relative Error"] + 
                            evaluation["KS Statistic"] + 
                            (1 - evaluation["Correlation"]))
                    
                    if score < best_score:
                        best_score = score
                        best_params = (weights, alphas, betas, n_components)
                        best_evaluation = evaluation
                        self.tail_data = tail_data
                        self.threshold = threshold

                except Exception as e:
                    print(f"Threshold {threshold} fitting failed: {str(e)}")
                    continue

            if best_params is None:
                # 如果所有拟合都失败，使用简单的单组件拟合
                self.weights = np.array([1.0])
                self.alphas = np.array([2.0])
                self.betas = np.array([np.mean(self.original_data)])
                self.n_components = 1
                self.threshold = 99
                self.tail_data = sorted_data[sorted_data > np.percentile(sorted_data, 99)]
                return self, {"Verdict": "Fallback to simple fit"}

            self.weights, self.alphas, self.betas, self.n_components = best_params
            print(self.weights, self.alphas, self.betas, self.n_components)
            return self, best_evaluation

        except Exception as e:
            print(f"Fitting failed: {str(e)}")
            return self, {"Verdict": "Fitting failed"}
    def extract_data(self, X):
        try:
            # 计算方差
            variances = np.var(X, axis=0, ddof=1)
            
            # 如果是稀疏矩阵，转换为数组
            if sp.issparse(variances):
                variances = variances.A1
                
            # 处理可能的nan值
            variances = np.nan_to_num(variances, nan=np.nanmean(variances))
            
            # 确保所有方差都是正数且不为零
            variances = np.maximum(variances, 1e-10)
            
            return variances
            
        except Exception as e:
            print(f"Error in extract_data: {str(e)}")
            return None
        
    def assess_tail_discreteness(self, tail_data):
    
        try:
            # 对数据进行排序
            sorted_tail = np.sort(tail_data)
            
            # 计算相邻值之间的差异
            differences = np.diff(sorted_tail)
            
            # 计算变异系数 (CV = 标准差/平均值)
            if np.mean(differences) == 0:
                return 'discrete'
                
            cv = np.std(differences) / np.mean(differences)
            
            # 根据变异系数判断离散程度
            if cv > 2.0:
                return 'discrete'  # 高变异表示离散
            elif cv < 1.0:
                return 'smooth'   # 低变异表示平滑
            else:
                return 'mixed'    # 介于之间表示混合
                
        except Exception as e:
            print(f"Error in assess_tail_discreteness: {str(e)}")
            return 'discrete'  # 发生错误时默认返回离散
    def fit_and_simulate(self, adata, n_iterations=10):  # 减少迭代次数
        """多次拟合和模拟，选择最佳结果"""
        best_simulation = None
        best_evaluation = None
        best_score = float('inf')
        best_params = None

        for _ in range(n_iterations):
            self.random_state = np.random.RandomState()
            try:
                self, evaluation = self.fit(adata)
                if evaluation.get("Verdict") == "Fitting failed":
                    continue
                    
                simulated_values = self.simulate(len(self.original_data))
                final_evaluation = self.evaluate_fit(self.original_data, simulated_values)

                score = (abs(final_evaluation["Cohen's d"]) + 
                        final_evaluation["Relative Error"] + 
                        final_evaluation["KS Statistic"] + 
                        (1 - final_evaluation["Correlation"]))

                if score < best_score:
                    best_score = score
                    best_simulation = simulated_values
                    best_evaluation = final_evaluation
                    best_params = {
                        'weights': self.weights.copy(),
                        'alphas': self.alphas.copy(),
                        'betas': self.betas.copy(),
                        'n_components': self.n_components,
                        'threshold': self.threshold,
                        'tail_data': self.tail_data.copy() if self.tail_data is not None else None
                    }

            except Exception as e:
                continue

        if best_params is not None:
            self.weights = best_params['weights']
            self.alphas = best_params['alphas']
            self.betas = best_params['betas']
            self.n_components = best_params['n_components']
            self.threshold = best_params['threshold']
            self.tail_data = best_params['tail_data']
            return best_simulation, best_evaluation
        else:
            print("Warning: No successful fit found in any iteration")
            return None, {"Verdict": "All iterations failed"}

    def simulate(self, n_samples):
        """优化后的模拟方法"""
        try:
            n_main = int(n_samples * self.threshold / 100)
            n_tail = n_samples - n_main

            # 一次性生成所有样本
            main_samples = np.zeros(n_main)
            start_idx = 0
            for i in range(self.n_components):
                n_comp = int(n_main * self.weights[i])
                if i == self.n_components - 1:  # 最后一个组件
                    n_comp = n_main - start_idx
                main_samples[start_idx:start_idx + n_comp] = invgamma.rvs(
                    self.alphas[i], 
                    scale=self.betas[i], 
                    size=n_comp, 
                    random_state=self.random_state
                )
                start_idx += n_comp

            # 高效处理尾部数据
            tail_type = self.assess_tail_discreteness(self.tail_data)
            if tail_type == 'discrete':
                new_tail = self.random_state.choice(self.tail_data, size=n_tail, replace=True)
            else:
                # 使用线性插值
                new_tail = np.interp(
                    np.linspace(0, 1, n_tail),
                    np.linspace(0, 1, len(self.tail_data)),
                    np.sort(self.tail_data)
                )

            # 合并并排序
            new_samples = np.concatenate([main_samples, new_tail])
            new_samples = np.clip(new_samples, np.min(self.original_data), np.max(self.original_data))
            new_samples = np.sort(new_samples)

            # 恢复原始顺序
            simulated_data = np.zeros_like(new_samples)
            simulated_data[self.original_order] = new_samples

            return simulated_data

        except Exception as e:
            print(f"Simulation failed: {str(e)}")
            return self.random_state.choice(self.original_data, size=n_samples, replace=True)
        
    @staticmethod
    def evaluate_fit(original, generated, quantiles=[0.25, 0.5, 0.75, 0.9, 0.95, 0.99, 1]):
        def cohens_d(x1, x2):
            n1, n2 = len(x1), len(x2)
            var1, var2 = np.var(x1, ddof=1), np.var(x2, ddof=1)
            pooled_se = np.sqrt(((n1 - 1) * var1 + (n2 - 1) * var2) / (n1 + n2 - 2))
            return (np.mean(x1) - np.mean(x2)) / pooled_se

        def relative_error(x1, x2):
            return np.abs(np.mean(x1) - np.mean(x2)) / np.mean(x1)

        effect_size = cohens_d(original, generated)
        rel_error = relative_error(original, generated)
        ks_stat, _ = ks_2samp(original, generated)
        correlation = np.corrcoef(np.sort(original), np.sort(generated))[0, 1]

        orig_quant = np.quantile(original, quantiles)
        gen_quant = np.quantile(generated, quantiles)
        quant_rel_errors = np.abs(orig_quant - gen_quant) / orig_quant

        results = {
            "Cohen's d": effect_size,
            "Relative Error": rel_error,
            "KS Statistic": ks_stat,
            "Correlation": correlation,
            "Quantile Relative Errors": dict(zip([f"{q*100}th" for q in quantiles], quant_rel_errors))
        }

        excellent = (abs(effect_size) < 0.05 and rel_error < 0.05 and ks_stat < 0.1 and correlation > 0.95)
        good = (abs(effect_size) < 0.1 and rel_error < 0.15 and ks_stat < 0.15 and correlation > 0.9)
        fair = (abs(effect_size) < 0.2 and rel_error < 0.2 and ks_stat < 0.2 and correlation > 0.8)

        if excellent:
            verdict = "Excellent fit"
        elif good:
            verdict = "Good fit"
        elif fair:
            verdict = "Fair fit"
        else:
            verdict = "Poor fit"

        results["Verdict"] = verdict

        return results
    
def simulate_gene_variances_advanced(adata, n_iterations=10):
    simulator = IG_VarianceSimulator()
    simulated_values, final_evaluation = simulator.fit_and_simulate(adata, n_iterations)
    
    result_dict = dict(zip(adata.var_names, simulated_values))
    return result_dict, final_evaluation

In [13]:
def run_simulation_domain(adata):
    adata.var_names_make_unique()
    adata.obs_names_make_unique()
    sc.pp.calculate_qc_metrics(adata, inplace=True)

    all_genes = adata.var_names.tolist()
    print(adata)

    # 模拟均值
    # simulated_means, mean_threshold, mean_evaluation = simulate_gene_means(adata, mode=mode, threshold=threshold)
    # print(f"Mean simulation - Best threshold: {mean_threshold}")
    # print(f"Mean simulation - Evaluation: {mean_evaluation}")

    simulated_means = simulate_gene_average_expression(adata)
    # 模拟方差
    simulated_vars, var_evaluation= simulate_gene_variances_advanced(adata)
    

    print(f"Variance simulation - Evaluation: {var_evaluation}")

    print(f"Number of simulated genes: {len(simulated_means)}")
    # 使用模拟的参数拟合边际模型
    model_params = fit_marginal_model_with_simulated_params(
        adata, 
        simulated_means, 
        simulated_vars, 
        min_nonzero_num=2, 
        maxiter=500, 
        n_jobs=-1
    )

    # 添加模拟评估结果到返回的字典中
    model_params['simulation_evaluation'] = {
        'variance': var_evaluation,

    }

    return model_params

In [15]:
adata = sc.read_h5ad("/Users/chen_yiru/Desktop/simulation/data/raw/processed_151673_filtered.h5ad")
sc.pp.filter_genes(adata, min_cells=50)
adata.write_h5ad("/Users/chen_yiru/Desktop/simulation/data/raw/processed_151673_filtered.h5ad")

In [ ]:
def combine_models(global_params, domain_params, alpha=0.3):
    combined_params = {}
    
    # 拷贝通用的参数 ('gene_sel1', 'gene_sel2')
    for key in global_params:
        if key in ['gene_sel1', 'gene_sel2']:
            combined_params[key] = global_params[key]
        elif key == 'marginal_param1':  # 处理 'marginal_param1' 参数
            global_genes = global_params['gene_sel1'].values()
            domain_genes = domain_params['gene_sel1'].values()
            
            combined_marginal_param1 = []
            for i, gene in enumerate(global_genes):
                if gene in domain_genes:  # 如果域模型中也有这个基因
                    domain_idx = list(domain_genes).index(gene)
                    # 使用全局和域的参数加权平均
                    combined_param = alpha * np.array(global_params[key][i]) + (1 - alpha) * np.array(domain_params[key][domain_idx])
                else:  # 如果域模型没有这个基因，直接使用全局的参数
                    combined_param = global_params[key][i]
                
                # 确保参数在合理的范围内
                combined_param[0] = min(max(combined_param[0], 0), 1)  # pi 的范围是 [0, 1]
                combined_param[1] = max(combined_param[1], 1e-8)       # r 的最小值 1e-8，防止负值
                combined_param[2] = max(combined_param[2], 1e-8)       # mu 的最小值 1e-8
                
                combined_marginal_param1.append(combined_param)
            
            combined_params[key] = combined_marginal_param1
        else:
            combined_params[key] = global_params[key]
    
    # 如果模型类型发生冲突，优先使用 domain 模型参数，避免使用无效的全局参数
    for i, (global_model, domain_model) in enumerate(zip(global_params['model_selected'], domain_params['model_selected'])):
        if global_model != domain_model:
            print(f"Warning: Inconsistent model types for gene {i}. Using domain model {domain_model}.")

            # 以 domain 模型为准
            combined_params['model_selected'][i] = domain_model

            # 检查全局参数是否无效，如果无效就不进行修正，直接使用 domain 参数
            global_param = global_params['marginal_param1'][i]
            domain_param = domain_params['marginal_param1'][i]

            if domain_model == 'Poisson':
                # 修正 mu，前提是全局参数有效
                global_mu = global_param[2]
                if not np.isinf(global_mu) and not np.isnan(global_mu):
                    combined_params['marginal_param1'][i][2] = alpha * global_mu + (1 - alpha) * domain_param[2]
                else:
                    combined_params['marginal_param1'][i][2] = domain_param[2]

            elif domain_model == 'NB':
                # 修正 r 和 mu，前提是全局参数有效
                global_r = global_param[1]
                global_mu = global_param[2]

                if not np.isinf(global_r) and not np.isnan(global_r):
                    combined_params['marginal_param1'][i][1] = alpha * global_r + (1 - alpha) * domain_param[1]
                else:
                    combined_params['marginal_param1'][i][1] = domain_param[1]
                
                if not np.isinf(global_mu) and not np.isnan(global_mu):
                    combined_params['marginal_param1'][i][2] = alpha * global_mu + (1 - alpha) * domain_param[2]
                else:
                    combined_params['marginal_param1'][i][2] = domain_param[2]

            elif domain_model == 'ZIP':
                # 修正 pi 和 mu，前提是全局参数有效
                global_pi = global_param[0]
                global_mu = global_param[2]

                if not np.isinf(global_pi) and not np.isnan(global_pi):
                    combined_params['marginal_param1'][i][0] = alpha * global_pi + (1 - alpha) * domain_param[0]
                else:
                    combined_params['marginal_param1'][i][0] = domain_param[0]
                
                if not np.isinf(global_mu) and not np.isnan(global_mu):
                    combined_params['marginal_param1'][i][2] = alpha * global_mu + (1 - alpha) * domain_param[2]
                else:
                    combined_params['marginal_param1'][i][2] = domain_param[2]

            elif domain_model == 'ZINB':
                # 修正 pi, r 和 mu，前提是全局参数有效
                global_pi = global_param[0]
                global_r = global_param[1]
                global_mu = global_param[2]

                if not np.isinf(global_pi) and not np.isnan(global_pi):
                    combined_params['marginal_param1'][i][0] = alpha * global_pi + (1 - alpha) * domain_param[0]
                else:
                    combined_params['marginal_param1'][i][0] = domain_param[0]

                if not np.isinf(global_r) and not np.isnan(global_r):
                    combined_params['marginal_param1'][i][1] = alpha * global_r + (1 - alpha) * domain_param[1]
                else:
                    combined_params['marginal_param1'][i][1] = domain_param[1]
                
                if not np.isinf(global_mu) and not np.isnan(global_mu):
                    combined_params['marginal_param1'][i][2] = alpha * global_mu + (1 - alpha) * domain_param[2]
                else:
                    combined_params['marginal_param1'][i][2] = domain_param[2]

    return combined_params

adata = adata[~adata.obs['Ground Truth'].isna()].copy()


unique_ground_truths = adata.obs['Ground Truth'].unique()


global_model_params = run_simulation_tissue(adata)


split_adatas = {}
domain_params = {}

for ground_truth in unique_ground_truths:
    
    mask = adata.obs['Ground Truth'] == ground_truth
    
    # 创建新的 AnnData 对象
    new_adata = ad.AnnData(
        X=adata[mask].X.copy(),
        obs=adata[mask].obs.copy(),
        var=adata.var.copy(),
        uns=adata.uns.copy(),
        obsm=adata[mask].obsm.copy() if adata.obsm is not None else None,
        varm=adata.varm.copy() if adata.varm is not None else None,
        layers=adata[mask].layers.copy() if adata.layers is not None else None
    )
    
    
    new_adata.obs_names_make_unique()
    
    split_adatas[ground_truth] = new_adata
    
    # 拟合 domain 特定模型
    domain_params[ground_truth] = run_simulation_domain(new_adata)

    print(f"Ground Truth: {ground_truth}")
    print(f"Number of cells: {new_adata.shape[0]}")
    print(f"Number of genes: {new_adata.shape[1]}")

simulated_adatas = {}
alpha = 0.5

for ground_truth, split_adata in split_adatas.items():
    print(f"Simulating Ground Truth: {ground_truth}")
  
    combined_params = combine_models(global_model_params, domain_params[ground_truth], alpha)
    simsrt = SimSRT(split_adata, combined_params)
    
    simulated_simsrt = srtsim_remain_simulate_count(simsrt, split_adata, num_cores=8, verbose=True)
    simulated_counts = simulated_simsrt.simCounts
    
    if simulated_counts.shape != split_adata.shape:
        print(f"Warning: simulated_counts shape {simulated_counts.shape} does not match adata shape {split_adata.shape}")
        if simulated_counts.shape == (split_adata.shape[1], split_adata.shape[0]):
            simulated_counts = simulated_counts.T
        elif simulated_counts.shape != split_adata.shape:
            raise ValueError("Cannot adjust simulated_counts shape to match adata shape")
    
    simulated_adata = ad.AnnData(
        X=simulated_counts,
        obs=split_adata.obs.copy(),
        var=split_adata.var.copy(),
        obsm={'spatial': split_adata.obsm['spatial']}
    )

    simulated_adata.obs['total_counts'] = simulated_adata.X.sum(axis=1)
    simulated_adata.obs['n_genes'] = (simulated_adata.X > 0).sum(axis=1)

    simulated_adatas[ground_truth] = simulated_adata


merged_simulated_adata = ad.concat(
    list(simulated_adatas.values()),
    axis=0,
    join='outer',
    merge='same',
    label='Ground Truth',
    keys=list(simulated_adatas.keys())
)


merged_simulated_adata.obs_names_make_unique()

print("Merged Simulated AnnData:")
print(merged_simulated_adata)
print("\nGround Truth distribution:")
print(merged_simulated_adata.obs['Ground Truth'].value_counts())

AnnData object with n_obs × n_vars = 3553 × 13072
    obs: 'Ground Truth', 'n_counts', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes'
    var: 'gene_ids', 'feature_types', 'genome', 'n_cells', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts'
    obsm: 'spatial'
[1.] [1.1] [0.06676248] 1
[1.] [1.1] [0.0632476] 1
[1.] [1.1] [0.06676248] 1
[1.] [1.1] [0.0632476] 1
[1.] [1.1] [0.06056084] 1
[1.] [1.1] [0.06676248] 1
[1.] [1.1] [0.05941892] 1
[1.] [1.1] [0.06676248] 1
[1.] [1.1] [0.06056084] 1
[1.] [1.1] [0.06676248] 1
Variance simulation - Evaluation: {"Cohen's d": -0.0005740421480767003, 'Relative Error': 0.020187290177713767, 'KS Statistic': 0.13310893512851896, 'Correlation': 0.9835400223812112, 'Quantile Relative Errors': {'25.0th': 0.1250588361494914, '50.0t

Fitting models: 100%|██████████| 13072/13072 [00:15<00:00, 863.87it/s] 


AnnData object with n_obs × n_vars = 988 × 13072
    obs: 'Ground Truth', 'n_counts', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes'
    var: 'gene_ids', 'feature_types', 'genome', 'n_cells', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts'
    obsm: 'spatial'
[1.] [1.1] [0.05969268] 1
[1.] [1.1] [0.06192278] 1
[1.] [1.1] [0.05969268] 1
[1.] [1.1] [0.06473195] 1
[1.] [1.1] [0.05969268] 1
[1.] [1.1] [0.05873957] 1
[1.] [1.1] [0.06473195] 1
[1.] [1.1] [0.05969268] 1
[1.] [1.1] [0.05969268] 1
[1.] [1.1] [0.06473195] 1
Variance simulation - Evaluation: {"Cohen's d": -0.00046112815880224327, 'Relative Error': 0.016005464045839562, 'KS Statistic': 0.17120563035495717, 'Correlation': 0.9820639169133482, 'Quantile Relative Errors': {'25.0th': 0.24243494403805627, '50

Fitting models: 100%|██████████| 13072/13072 [00:01<00:00, 7735.65it/s]


Ground Truth: Layer_3
Number of cells: 988
Number of genes: 13072
AnnData object with n_obs × n_vars = 243 × 13072
    obs: 'Ground Truth', 'n_counts', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes'
    var: 'gene_ids', 'feature_types', 'genome', 'n_cells', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts'
    obsm: 'spatial'
[1.] [2.] [0.07826029] 1
[1.] [2.] [0.07826029] 1


In [30]:
merged_simulated_adata.write_h5ad("/Users/chen_yiru/Desktop/simulation/data/simulated/domain_156763.h5ad")

## 扰动模拟测试

In [ ]:
import numpy as np
import anndata



def add_perturbations(model_params, perturbation_scale=0.1):
    perturbed_model_params = model_params.copy()
    perturbed_marginal_param1 = []
    for params, model_type in zip(model_params['marginal_param1'], model_params['model_selected']):
        perturbation_factor = 1 + perturbation_scale * np.random.choice([-1, 1])
        
        if model_type == "Poisson":
            _, _, mu = params
            mu_perturbed = max(1e-6, mu * perturbation_factor)
            perturbed_marginal_param1.append([0, np.inf, mu_perturbed])

        elif model_type == "NB":
            _, theta, mu = params
            theta_perturbed = max(1e-6, theta * perturbation_factor)
            mu_perturbed = max(1e-6, mu * perturbation_factor)
            perturbed_marginal_param1.append([0, theta_perturbed, mu_perturbed])

        elif model_type == "ZIP":
            pi0, _, mu = params
            pi0_perturbed = max(0, min(1, pi0 * perturbation_factor))
            mu_perturbed = max(1e-6, mu * perturbation_factor)
            perturbed_marginal_param1.append([pi0_perturbed, np.inf, mu_perturbed])
        elif model_type == "ZINB":
            pi0, theta, mu = params
            pi0_perturbed = max(0, min(1, pi0 * perturbation_factor))
            theta_perturbed = max(1e-6, theta * perturbation_factor)
            mu_perturbed = max(1e-6, mu * perturbation_factor)
            perturbed_marginal_param1.append([pi0_perturbed, theta_perturbed, mu_perturbed])

    perturbed_model_params['marginal_param1'] = perturbed_marginal_param1

    return perturbed_model_params


def run_simulation_with_perturbation(adata, model_params, perturbation_scale, output_dir):
    model_params = run_simulation_tissue(adata)
    perturbed_model_params = add_perturbations(model_params, perturbation_scale)
    simsrt = SimSRT(adata, perturbed_model_params)
    simulated_simsrt_noise = srtsim_remain_simulate_count(simsrt, adata, num_cores=8, verbose=True)
    simulated_counts = simulated_simsrt_noise.simCounts

    simulated_noise_adata = anndata.AnnData(
        X=simulated_counts.T,
        obs=adata.obs.copy(),
        var=adata.var.copy(),
        obsm={'spatial': adata.obsm['spatial']}
    )

    simulated_noise_adata.obs['total_counts'] = simulated_noise_adata.X.sum(axis=1)
    simulated_noise_adata.obs['n_genes'] = (simulated_noise_adata.X > 0).sum(axis=1)
    output_path = f"{output_dir}/simulated_noise_{perturbation_scale}.h5ad"
    simulated_noise_adata.write_h5ad(output_path)
    print(f"Saved perturbed data with scale {perturbation_scale} to {output_path}")


adata = sc.read_h5ad("/Users/chen_yiru/Desktop/simulation/Sample_data_151676.h5ad")
output_dir = "/Users/chen_yiru/Desktop/simulation/results"
perturbation_scales = [0.15]

# 运行不同扰动尺度的模拟
for scale in perturbation_scales:
    run_simulation_with_perturbation(adata, model_params, scale, output_dir)

## Reference free 新测试